In [1]:
import numpy as np
import torch
import glob
import os

from sar_transformer.dataset import *
from config import EnvironmentConfig, TransformerModelConfig
from models.trajectory_transformer import AlgorithmDistillationTransformer
from generation import *

pygame 2.3.0 (SDL 2.24.2, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
dataset = HistoryDataset("histories")
train_loader = create_history_dataloader(dataset, 128, 512*128)

In [3]:
context_len = dataset.n_episodes_per_seq * dataset.episode_length * 3 - 2
env = DarkKeyDoor(12, 2, 12, seed=500)

In [4]:
environment_config = EnvironmentConfig(
    env_id="Graph_DarkKeyDoor",
    env=env,
    device="cuda")

transformer_model_config = TransformerModelConfig(
    d_model=64,
    n_heads=4,
    d_mlp=256,
    n_layers=4,
    layer_norm=True,
    time_embedding_type="embedding",
    state_embedding_type="linear",
    n_ctx=context_len,
    device="cuda",
)

model = AlgorithmDistillationTransformer(environment_config, transformer_model_config)

In [5]:
model

AlgorithmDistillationTransformer(
  (action_embedding): Sequential(
    (0): Embedding(3, 64)
  )
  (time_embedding): Embedding(13, 64)
  (state_embedding): Linear(in_features=12, out_features=64, bias=False)
  (transformer): HookedTransformer(
    (embed): Identity()
    (hook_embed): HookPoint()
    (pos_embed): PosEmbedTokens()
    (hook_pos_embed): HookPoint()
    (blocks): ModuleList(
      (0-3): 4 x TransformerBlock(
        (ln1): LayerNorm(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (ln2): LayerNorm(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (attn): Attention(
          (hook_k): HookPoint()
          (hook_q): HookPoint()
          (hook_v): HookPoint()
          (hook_z): HookPoint()
          (hook_attn_scores): HookPoint()
          (hook_pattern): HookPoint()
          (hook_result): HookPoint()
        )
        (mlp): MLP(
          (hook_pre): HookPoint()
          

In [6]:
from sar_transformer.trainer import train
train(model, train_loader, env)

Training AD, Epoch 46: 0.2730:  97%|█████████▋| 498/512 [01:22<00:02,  5.90it/s]